In [1]:
import re
import warnings
import pandas as pd
from bs4 import BeautifulSoup

warnings.filterwarnings("ignore", category=UserWarning, module="bs4")
pd.set_option("display.max_rows", None)

In [2]:
df_gugun = pd.read_csv("input/자치구.csv")
df_gugun

,자치구,행정동코드
0,종로구,1111000000
1,중구,1114000000
2,용산구,1117000000
3,성동구,1120000000
4,광진구,1121500000
5,동대문구,1123000000
6,중랑구,1126000000
7,성북구,1129000000
8,강북구,1130500000
9,도봉구,1132000000


In [3]:
df_origin = pd.read_csv("input/생태문화길 코스정보.csv")[
  ["순번", "자치구", "거리", "소요시간", "연계지하철역", "교통편", "상세코스", "설명", "코스명", "강남강북구분", "코스레벨"]
].sort_values(by=["순번"], axis=0).reset_index(drop=True)
df_origin

,순번,자치구,거리,소요시간,연계지하철역,교통편,상세코스,설명,코스명,강남강북구분,코스레벨
0,1,강서구,2.51km,1시간30분,9호선,진입로 1 : 양천향교역 지하철 9호선 2번 출입구<br /> ※교통편의 변경이 있...,양천향교~겸재정선기념관~구암공원~허준박물관,강서구의 명소들을 차례로 들르며 걷게 되는 코스 가장 먼저 서울 유일의 향교인 양...,강서문화 산책길,강남권,초급
1,2,성북구,2.20km,1시간30분,"4호선,6호선",진입로 1 : 길음역 지하철 4호선 2번 출구 <br /> 진입로 2 : 통일교회 ...,개운초등학교~성북구의회~고려대 북문,대도시에서 “녹색허파”로서의 역할과 수도권 주민들의 자연휴식처로 크게 애용되고 있...,개운산 숲 나들길,강북권,초급
2,3,구로구,1.50km,1시간,1호선,진입로 1 : 오류동역 지하철 1호선 1번 출구 <br /> 진입로 2 :...,배드민턴장~야생화단지~숲속길,개웅산은 능선이 세 방향으로 뻗어 내려 삼각형 모양을 이루고 있는 산으로서 산 전...,개웅산 숲 산책길,강남권,초급
3,4,강서구,7.74km,2시간40분,5호선,진입로 1 : 개화산역 지하철 5호선 2번 출구 <br /> 진입로 2 ~...,하늘길 전망대~개화산 호국충혼비~약사사,개화산에서 시작하는 이 길은 오솔길과 여러 공원들이 각기 특별한 매력을 발산하며 ...,"개화산 나들길(강서 둘레길 1,2코스)",강남권,중급
4,5,강동구,1.90km,1시간30분,5호선,진입로1 : 명일역 지하철 5호선 3번 출입구 <br /> ※교통편의 변경이 있을 ...,명일역 3번 출입구~고덕산 정상,해발 100m 미만의 낮고 순탄한 숲길이지만 계절 따라 순환하는 자연의 섭리를 그...,고덕산 산책길,강남권,초급
5,6,"관악구,금천구",5.58km,2시간,2호선,"진입로 1 : 법대입구 버스 정류장(21-298) 지선버스 5511,5516 <br...",관악산호수공원~용천수~관악산야외식물원,관악산 계곡길은 사시사철 언제라도 시원한 물줄기를 따라 길을 잇는다. 관악산 등산...,관악산 계곡 나들길,강남권,중급
6,7,"관악구,금천구",6.99km,3시간,NaN,진입로 1 : 신동아아파트 버스 정류장(21-326) 지선버스 5515 <br />...,샘말공원~맨발공원~약수암~신우초등학교,예전부터 고시원이 밀집된 대학동 일대를 걸을 수 있도록 기 조성된 공원들을 연결하...,관악산 나들길,강남권,중급
7,8,"관악구,금천구",0.81km,40분,NaN,"진입로 : 사당역 지하철 2, 4호선 4번 출입구",관음사~상록약수터,오래된 나무와 온갖 풀이 바위와 어우러져 철따라 변하는 모습이 마치 금강산과 같다...,관음사 산책길,강남권,초급
8,9,구로구,6.90km,3시간,"1호선,7호선",진입로 1 ~ 3 : 도보진입 <br /> 진입로 2 : 신정이펜하우스1단...,지양산 숲길~차돌바위~궁동터널 위 지양산 숲길 시작,위성사진을 통해 본 지양산 일대는 서울 남서쪽에 동동 떠 있는 커다란 섬이다. 삭막...,구로 지양산 숲 나들길,강남권,중급
9,10,강남구,6.02km,2시간30분,3호선,진입로 1 : 매봉역 지하철 3호선 4번 출구 <br /> 진입로 2 :...,달터근린공원~구룡산길 시작~개암약수터,구룡산은 옛날 임신한 여인이 용 열 마리가 승천하는 것을 보고 놀라 소리치는 바람...,구룡산 나들길,강남권,중급


In [4]:
def convert_location_names_to_codes(names: str) -> str:
  name_list = names.split(",")
  code_list = []

  for name in name_list:
    code_list.append(df_gugun[df_gugun.자치구 == name].iloc[0].행정동코드)

  code_list.sort()
  return ",".join(map(str, code_list))

In [5]:
df = pd.DataFrame()
df["id"] = df_origin.순번
df["distance"] = df_origin.거리.map(lambda x: int(float(x.lower().replace("㎞", "km").split("km")[0]) * 1000))
df["time"] = df_origin.소요시간.map(lambda x: (
  int(x.split("시간")[0].strip() if "시간" in x else 0) * 60
  + int(x.split("시간")[-1].split("분")[0].strip() if "분" in x else 0)
))
df["near_subway"] = df_origin.연계지하철역.fillna("").map(lambda x: x.replace(",", ", ").strip())
df["access_way"] = df_origin.교통편.map(lambda x: re.sub(" +", " ", BeautifulSoup(x, "html5lib").get_text(separator="\n", strip=True).strip()))
df["route"] = df_origin.상세코스.map(lambda x: x.strip())
df["description"] = df_origin.설명.map(lambda x: re.sub(" +", " ", BeautifulSoup(x, "html5lib").get_text(separator="\n", strip=True).strip()))
df["name"] = df_origin.코스명.map(lambda x: x.strip())
df["region"] = df_origin.강남강북구분.replace(["강남권", "강북권"], [1, 2])
df["level"] = df_origin.코스레벨.replace(["초급", "중급", "고급"], [1, 2, 3])
df

,id,distance,time,near_subway,access_way,route,description,name,region,level
0,1,2510,90,9호선,진입로 1 : 양천향교역 지하철 9호선 2번 출입구\n※교통편의 변경이 있을 수 있...,양천향교~겸재정선기념관~구암공원~허준박물관,강서구의 명소들을 차례로 들르며 걷게 되는 코스 가장 먼저 서울 유일의 향교인 양천...,강서문화 산책길,1,1
1,2,2200,90,"4호선, 6호선",진입로 1 : 길음역 지하철 4호선 2번 출구\n진입로 2 : 통일교회 버스 정류장...,개운초등학교~성북구의회~고려대 북문,대도시에서 “녹색허파”로서의 역할과 수도권 주민들의 자연휴식처로 크게 애용되고 있다...,개운산 숲 나들길,2,1
2,3,1500,60,1호선,진입로 1 : 오류동역 지하철 1호선 1번 출구\n진입로 2 : 개봉한진아파트 버스...,배드민턴장~야생화단지~숲속길,개웅산은 능선이 세 방향으로 뻗어 내려 삼각형 모양을 이루고 있는 산으로서 산 전체...,개웅산 숲 산책길,1,1
3,4,7740,160,5호선,진입로 1 : 개화산역 지하철 5호선 2번 출구\n진입로 2 ~ 5 : 도보접근\n...,하늘길 전망대~개화산 호국충혼비~약사사,개화산에서 시작하는 이 길은 오솔길과 여러 공원들이 각기 특별한 매력을 발산하며 독...,"개화산 나들길(강서 둘레길 1,2코스)",1,2
4,5,1900,90,5호선,진입로1 : 명일역 지하철 5호선 3번 출입구\n※교통편의 변경이 있을 수 있으니 ...,명일역 3번 출입구~고덕산 정상,해발 100m 미만의 낮고 순탄한 숲길이지만 계절 따라 순환하는 자연의 섭리를 그대...,고덕산 산책길,1,1
5,6,5580,120,2호선,"진입로 1 : 법대입구 버스 정류장(21-298) 지선버스 5511,5516\n(서...",관악산호수공원~용천수~관악산야외식물원,관악산 계곡길은 사시사철 언제라도 시원한 물줄기를 따라 길을 잇는다. 관악산 등산로...,관악산 계곡 나들길,1,2
6,7,6990,180,,진입로 1 : 신동아아파트 버스 정류장(21-326) 지선버스 5515\n- 지하철...,샘말공원~맨발공원~약수암~신우초등학교,예전부터 고시원이 밀집된 대학동 일대를 걸을 수 있도록 기 조성된 공원들을 연결하여...,관악산 나들길,1,2
7,8,810,40,,"진입로 : 사당역 지하철 2, 4호선 4번 출입구",관음사~상록약수터,오래된 나무와 온갖 풀이 바위와 어우러져 철따라 변하는 모습이 마치 금강산과 같다 ...,관음사 산책길,1,1
8,9,6900,180,"1호선, 7호선",진입로 1 ~ 3 : 도보진입\n진입로 2 : 신정이펜하우스1단지 버스 정류장 마을...,지양산 숲길~차돌바위~궁동터널 위 지양산 숲길 시작,위성사진을 통해 본 지양산 일대는 서울 남서쪽에 동동 떠 있는 커다란 섬이다. 삭막...,구로 지양산 숲 나들길,1,2
9,10,6020,150,3호선,진입로 1 : 매봉역 지하철 3호선 4번 출구\n진입로 2 : 능인선원앞 버스 정류...,달터근린공원~구룡산길 시작~개암약수터,구룡산은 옛날 임신한 여인이 용 열 마리가 승천하는 것을 보고 놀라 소리치는 바람에...,구룡산 나들길,1,2


In [6]:
df_course_guguns = pd.DataFrame()
df_course_guguns["id"] = df_origin.순번
df_course_guguns["location_code"] = df_origin.자치구.map(convert_location_names_to_codes).str.split(",")
df_course_guguns = df_course_guguns.explode("location_code").reset_index(drop=True)
df_course_guguns

,id,location_code
0,1,1150000000
1,2,1129000000
2,3,1153000000
3,4,1150000000
4,5,1174000000
5,6,1154500000
6,6,1162000000
7,7,1154500000
8,7,1162000000
9,8,1154500000


In [7]:
df.to_csv("output/course-info.csv", index=False, quoting=1)
df_course_guguns.to_csv("output/course-info-guguns.csv", index=False, quoting=1)